# XGBoost

In [3]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

## Intro to XGBoost

## XGBoost Library

XGBoost is an industry-proven, open-source software library that provides a gradient boosting framework for scaling billions of data points quickly and efficiently.

Docs: https://xgboost.readthedocs.io/en/stable/index.html

**XGBoost** is an optimized distributed gradient boosting library designed to be highly **efficient, flexible and portable**. It implements machine learning algorithms under the Gradient Boosting framework. XGBoost provides a parallel tree boosting (also known as GBDT, GBM) that solve many data science problems in a fast and accurate way. The same code runs on major distributed environment (Hadoop, SGE, MPI) and can solve problems beyond billions of examples.

Installation: https://xgboost.readthedocs.io/en/stable/install.html#python
- `pip install xgboost`

In [1]:
import xgboost

print(xgboost.__version__)

1.7.4


## XGBoost native API

In [4]:
diamonds = sns.load_dataset("diamonds")
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [5]:
diamonds.shape

(53940, 10)

In [6]:
diamonds.describe()

,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


In [7]:
diamonds.describe(exclude=np.number)

,cut,color,clarity
count,53940,53940,53940
unique,5,7,8
top,Ideal,G,SI1
freq,21551,11292,13065


In [8]:
from sklearn.model_selection import train_test_split

# Extract feature and target arrays
X, y = diamonds.drop('price', axis=1), diamonds[['price']]

# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

# Convert to Pandas category
for col in cats:
    X[col] = X[col].astype('category')
    
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [9]:
import xgboost as xgb

# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [10]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "hist"} # gpu_hist -> grfična kartica

In [11]:
model = xgb.train(params=params, dtrain=dtrain_reg, num_boost_round=100)

In [12]:
from sklearn.metrics import mean_squared_error

preds = model.predict(dtest_reg)

In [13]:
rmse = mean_squared_error(y_test, preds, squared=False)
print(f"RMSE of the base model: {rmse:.3f}")

RMSE of the base model: 545.388


### Using Validation Sets During Training

In [14]:
params = {"objective": "reg:squarederror", "tree_method": "hist"}

In [15]:
evals = [(dtest_reg, "validation"), (dtrain_reg, "train")]

In [16]:
evals = [(dtest_reg, "validation"), (dtrain_reg, "train")]

model = xgb.train(params=params, dtrain=dtrain_reg, num_boost_round=100, evals=evals, verbose_eval=10) # Print every ten rounds

[0]	validation-rmse:3930.87087	train-rmse:3985.31595
[10]	validation-rmse:591.03042	train-rmse:557.19710
[20]	validation-rmse:550.76666	train-rmse:495.31647
[30]	validation-rmse:547.16647	train-rmse:467.13670
[40]	validation-rmse:544.10422	train-rmse:447.26879
[50]	validation-rmse:543.97371	train-rmse:432.51681
[60]	validation-rmse:544.77874	train-rmse:420.72943
[70]	validation-rmse:544.77491	train-rmse:408.72053
[80]	validation-rmse:544.33808	train-rmse:395.88816
[90]	validation-rmse:545.99682	train-rmse:383.62262
[99]	validation-rmse:545.38842	train-rmse:378.37454


### XGBoost Early Stopping

In [ ]:
model = xgb.train(params=params, dtrain=dtrain_reg, num_boost_round=5000, evals=evals,verbose_eval=500)

In [17]:
# If there’s more than one metric in the eval_metric parameter given in params, 
# the last metric will be used for early stopping.
evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=1000,
   evals=evals,
   verbose_eval=50,
   # Activate early stopping
   early_stopping_rounds=50
)

[0]	train-rmse:3985.31595	validation-rmse:3930.87087
[50]	train-rmse:432.51681	validation-rmse:543.97371
[87]	train-rmse:386.38896	validation-rmse:545.45681


### XGBoost Cross-Validation

In [18]:
params = {"objective": "reg:squarederror", "tree_method": "hist"}
n = 1000


results = xgb.cv(params, dtrain_reg, num_boost_round=n, nfold=5, early_stopping_rounds=20)

In [19]:
results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,3985.648654,10.343596,3986.913623,41.642778
1,2848.365726,8.014086,2851.020437,28.028733
2,2063.401458,4.637773,2068.629977,19.969459
3,1521.493751,3.874078,1530.496272,13.592330
4,1156.827103,2.991735,1170.413316,11.695597


In [20]:
best_rmse = results['test-rmse-mean'].min()
best_rmse

550.7196748119261

### XGBoost Classification

The two most popular classification objectives are:
- `binary:logistic` - binary classification (the target contains only two classes, i.e., cat or dog)
- `multi:softprob` - multi-class classification (more than two classes in the target, i.e., apple/orange/banana)

In [21]:
from sklearn.preprocessing import OrdinalEncoder

X, y = diamonds.drop("cut", axis=1), diamonds[['cut']]

# Encode y to numeric
y_encoded = OrdinalEncoder().fit_transform(y)

# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

# Convert to pd.Categorical
for col in cats:
    X[col] = X[col].astype('category')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, random_state=1, stratify=y_encoded)

In [22]:
y["cut"].unique()

['Ideal', 'Premium', 'Good', 'Very Good', 'Fair']
Categories (5, object): ['Ideal', 'Premium', 'Very Good', 'Good', 'Fair']

In [23]:
# Create classification matrices
dtrain_clf = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_clf = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [24]:
params = {"objective": "multi:softprob", "tree_method": "hist", "num_class": 5}

results = xgb.cv(
   params, dtrain_clf,
   num_boost_round=100,
   nfold=5,
   metrics=["mlogloss", "auc", "merror"],
   early_stopping_rounds=20
)

In [25]:
results.head()

,train-mlogloss-mean,train-mlogloss-std,train-auc-mean,train-auc-std,train-merror-mean,train-merror-std,test-mlogloss-mean,test-mlogloss-std,test-auc-mean,test-auc-std,test-merror-mean,test-merror-std
0,1.257340,0.000793,0.892373,0.000520,0.255772,0.000619,1.260954,0.001528,0.887060,0.001848,0.260042,0.002096
1,1.073036,0.000955,0.897153,0.000239,0.253918,0.000650,1.079572,0.002348,0.891436,0.001965,0.258139,0.002890
2,0.954985,0.001309,0.900132,0.000810,0.251533,0.002001,0.964390,0.003143,0.894313,0.001703,0.255617,0.001872
3,0.874229,0.001387,0.902673,0.000454,0.249951,0.001960,0.886645,0.003445,0.896172,0.002041,0.255172,0.002239
4,0.815914,0.002027,0.905565,0.001047,0.248999,0.002175,0.831286,0.003392,0.898215,0.001337,0.254283,0.002097


In [26]:
results['test-auc-mean'].max()

0.9387768185864125

## XGBoost Sklearn

In [27]:
from sklearn import datasets

X,y = datasets.load_diabetes(return_X_y=True)

In [28]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

In [29]:
scores = cross_val_score(XGBRegressor(objective='reg:squarederror'), X, y, scoring='neg_mean_squared_error')

In [30]:
(-scores)**0.5 

array([62.80101886, 65.78389959, 62.21211593, 66.40836809, 67.3001013 ])

In [31]:
from sklearn import datasets

X,y = datasets.load_breast_cancer(return_X_y=True)

In [32]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

cross_val_score(XGBClassifier(), X, y).mean()

0.9771619313771154